

__Set your own route planner for a given OSM map and GTFS file and query server on localhost for a detailed routes__


### OTP walkthrough

from: http://docs.opentripplanner.org/en/latest/Basic-Tutorial/

1. Install Java SDK
2. Download [latest](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/otp-1.4.0-shaded.jar) .jar from [Maven](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/)
3. Download gtfs.zip of a given city into a `folder` from `transit.land`
4. Download osm.pgf of your area from [here](https://www.interline.io/osm/extracts/) (need to login to get API token)
5. call from terminal `java -Xmx6G -jar otp-1.4.0-shaded.jar --build data_path --inMemory` changing `6G` as memory allocated (6GB) and `data_path` to your path with osm and gtfs files
6. if succesful you shall be able to open a web search in your browser: `http://localhost:8080/`
7. and query it via API, for instance: 'http://localhost:8080/otp/routers/default/plan?fromPlace=52.25177%2C20.92758&toPlace=52.20550%2C21.00517&time=1%3A02pm&date=04-09-2020&mode=TRANSIT%2CWALK&maxWalkDistance=5000&arriveBy=false'


In [1]:
import pandas as pd
import requests

local imports

In [2]:
from main import make_query, parse_OTP_response
from utils import plot

set up

In [3]:
PATH = 'georequests.csv'
OTP_API = "http://localhost:8080/otp/routers/default/plan"

input

In [4]:
df = pd.read_csv(PATH, index_col=[0])  # load the csv
df.treq = pd.to_datetime(df.treq)
df.sample(5)[['origin_x','origin_y','destination_x','destination_y','treq']]

,origin_x,origin_y,destination_x,destination_y,treq
780,4.873994,52.330740,4.860318,52.372202,2020-10-29 15:00:35
90,4.860563,52.361502,4.882519,52.371000,2020-10-29 15:15:19
608,4.835183,52.379731,4.880782,52.362382,2020-10-29 14:33:03
390,4.872908,52.376387,4.882440,52.370075,2020-10-29 14:58:01
94,4.879225,52.351436,4.878424,52.372836,2020-10-29 15:18:52


sample query (before you can query the server you need to run it -see run OTP server.ipynb in this folder)

[query synthax](http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html)  
[response json structure](http://dev.opentripplanner.org/apidoc/1.0.0/json_Response.html)

In [5]:
row = df.sample(1).squeeze()
query = make_query(row)
r = requests.get(OTP_API, params=query)
route = parse_OTP_response(r.json())
route

{'success': True,
 'n_itineraries': 3,
 'duration': 1757,
 'walkDistance': 1665.4487593870808,
 'transfers': 0,
 'transitTime': 420,
 'waitingTime': 2,
 'modes': [['WALK', 834, 1038], ['BUS', 420, 2447], ['WALK', 501, 626]]}

#### visualization

In [6]:
plot(r.json()['plan'], color = 'green')

Trip from (4.8341,52.3798) to (4.8718,52.3794) at 1603979040000. 
3 connections found. 
Best one is 29min (1665m walk, 0 transfer(s), wait time 0.03min)

LEG 	 MODE 	DIST 	TIME
-----------------------------
1	WALK	1039	834
2	BUS	2448	420
3	WALK	626	501


output

In [7]:
pd.read_csv('georequests_PT.csv', index_col=[0]).sample(5)

,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes
id,,,,,,,,
194,True,3,2126,965.962414,2,1199,165,"[['WALK', 304, 401], ['BUS', 540, 3182], ['WAL..."
241,True,3,2529,747.253113,0,1914,2,"[['WALK', 319, 385], ['TRAM', 1914, 7289], ['W..."
814,True,3,1817,875.008226,0,1080,2,"[['WALK', 98, 116], ['BUS', 1080, 4400], ['WAL..."
540,True,3,801,885.657564,0,94,2,"[['WALK', 573, 724], ['TRAM', 94, 447], ['WALK..."
650,True,3,1572,1076.375789,0,725,2,"[['WALK', 300, 368], ['TRAM', 725, 2576], ['WA..."


full response

In [8]:
r.json()

{'requestParameters': {'date': '10-29-2020',
  'mode': 'TRANSIT,WALK',
  'arriveBy': 'false',
  'fromPlace': '52.37983679999999,4.8341325',
  'toPlace': '52.3793529,4.871805',
  'time': '02:44pm',
  'maxWalkDistance': '2000'},
 'plan': {'date': 1603979040000,
  'from': {'name': 'Origin',
   'lon': 4.8341325,
   'lat': 52.37983679999999,
   'orig': '',
   'vertexType': 'NORMAL'},
  'to': {'name': 'Destination',
   'lon': 4.871805,
   'lat': 52.3793529,
   'orig': '',
   'vertexType': 'NORMAL'},
  'itineraries': [{'duration': 1757,
    'startTime': 1603979345000,
    'endTime': 1603981102000,
    'walkTime': 1335,
    'transitTime': 420,
    'waitingTime': 2,
    'walkDistance': 1665.4487593870808,
    'walkLimitExceeded': False,
    'elevationLost': 0.0,
    'elevationGained': 0.0,
    'transfers': 0,
    'legs': [{'startTime': 1603979345000,
      'endTime': 1603980179000,
      'departureDelay': 0,
      'arrivalDelay': 0,
      'realTime': False,
      'distance': 1038.839,
      'pa

---